<a href="https://colab.research.google.com/github/jyulkm/Tyler-Tech-Practicum/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from openai import OpenAI
import requests
import torch

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [3]:
data = pd.read_csv('HHS.csv', usecols=['Name', 'Description', 'Tags'])

In [5]:
data['name_description'] = data['Name'] + ' ' + data['Description']

In [6]:
data

,Name,Description,Tags,name_description
0,HAICViz - MuGSI,<p>The healthcare-associated infection compone...,"antimicrobial resistance,carbapenem-resistant ...",HAICViz - MuGSI <p>The healthcare-associated i...
1,HHS Data Governance Board Membership - w3i2-ja...,This dataset tracks the updates made on the da...,NaN,HHS Data Governance Board Membership - w3i2-ja...
2,Drug Interaction API - 5pe7-4rei - Archive Rep...,This dataset tracks the updates made on the da...,NaN,Drug Interaction API - 5pe7-4rei - Archive Rep...
3,NCHS Research and Development Survey (RANDS) R...,The Research and Development Survey (RANDS) is...,"gender identity,intimate partner violence,phys...",NCHS Research and Development Survey (RANDS) R...
4,"NNDSS - TABLE 1AA. Poliovirus infection, nonp...",This dataset tracks the updates made on the da...,NaN,"NNDSS - TABLE 1AA. Poliovirus infection, nonp..."
...,...,...,...,...
14454,HHS IDs,This file helps define the HHS_ID column that ...,"covid-19,hhs covid-19",HHS IDs This file helps define the HHS_ID colu...
14455,COVID-19 Nursing Home Dataset - ywg6-hgzb - Ar...,This dataset tracks the updates made on the da...,NaN,COVID-19 Nursing Home Dataset - ywg6-hgzb - Ar...
14456,Blog | The Health Consequences of Smoking—50 Y...,This blog post was posted by Boris Lushniak on...,blog,Blog | The Health Consequences of Smoking—50 Y...
14457,"Nutrition, Physical Activity, and Obesity - Be...","This dataset includes data on adult's diet, ph...","adults,brfss,dnpao,fruit,nutrition,obesity,ove...","Nutrition, Physical Activity, and Obesity - Be..."


In [7]:
data['name_description'].iloc[0]

'HAICViz - MuGSI <p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>'

In [13]:
client = OpenAI(api_key="sk-proj-OW3BIPRXsb4uhiwB1vO9T3BlbkFJNBekzWtxf16Wl2qBHmVS")

for i in range(3):
  completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[
      {"role": "system", "content": "You are a tag generating assistant, skilled in extracting key information based on description."},
      {"role": "user", "content": f"Generate 5 tags based on description: {data['name_description'].iloc[i]}"}
    ]
  )

  print('-------------------------------------------------')
  print(data['name_description'].iloc[i])
  print(completion.choices[0].message.content.replace('\n', ' '))

-------------------------------------------------
HAICViz - MuGSI <p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>
1. Healthcare-Associated Infections 2. CDC EIP Network 3. Antibiotic Resistance 4. Infection Tracking Methods 5. HAI Prevention Policies
-------------------------------------------------
HHS Data Governance Board Membership - w3i2-jayx - Archive Repository This dataset tracks the updates made on the dataset "HHS Data Governance Board 

In [9]:
completion.choices[0].message.content

ChatCompletion(id='chatcmpl-9JSoEiTwFuBoNsVxlsOqgran7oheg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. HAICViz\n2. MuGSI\n3. Healthcare-associated infection\n4. CDC EIP\n5. Antibiotic resistance', role='assistant', function_call=None, tool_calls=None))], created=1714426002, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=30, prompt_tokens=175, total_tokens=205))

In [11]:
completion.choices[0].message.content.replace('\n', ' ')

'1. HAICViz 2. MuGSI 3. Healthcare-associated infection 4. CDC EIP 5. Antibiotic resistance'

## Bart

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartForSequenceClassification

In [ ]:
# Load pretrained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Description input
description = data['Description'][0]

# Tokenize and preprocess the description
input_ids = tokenizer(description, return_tensors="pt", max_length=1024, truncation=True)["input_ids"]

# Generate tags
output = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
generated_tags = tokenizer.decode(output[0], skip_special_tokens=True)

# Postprocess tags if needed
# For example, split the generated tags into a list
generated_tags_list = generated_tags.split(',')

# Display or use generated tags
print("Generated Tags:", generated_tags_list)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Generated Tags: ['The healthcare-associated infection component of CDC’s EIP engages a network of state health departments and their academic medical center partners. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention']


In [ ]:
# Load pretrained BART model and tokenizer for sequence classification
model_name = "facebook/bart-large-mnli"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name)

# Define text input for classification
text = data['Description'][0]

# Tokenize and preprocess the text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted label
predicted_label_id = torch.argmax(outputs.logits, dim=1).item()

# Map label id to label name (if available)
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}  # Example mapping for MNLI task
predicted_label_name = label_map[predicted_label_id]

# Print predicted label
print("Predicted Label:", predicted_label_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Predicted Label: neutral


In [ ]:
data['Description'][0]

'<p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>'

In [ ]:
generated_tags

'The healthcare-associated infection component of CDC’s EIP engages a network of state health departments and their academic medical center partners. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention'